In [1]:
# import numpy as np
# import os
# import cv2
# import math
import pickle
from fr_utils import *
from inception_blocks import *
from keras import backend as K
K.set_image_data_format('channels_first')
# import tensorflow as tf
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# from numpy import genfromtxt
# from keras import backend as K
# K.set_image_data_format('channels_first')
# from keras.models import Model,Sequential
# from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
# from keras.layers.normalization import BatchNormalization
# from keras.layers.pooling import MaxPooling2D, AveragePooling2D
# from keras.layers.core import Lambda, Flatten, Dense


Using TensorFlow backend.


In [2]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    Implementation of the triplet loss as defined by formula (3)
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)
    
    Returns:
    loss -- real number, value of the loss
    """
       
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    # Step 1: Compute the (encoding) distance between the anchor and the positive, you will need to sum over axis=-1
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,positive)),axis=-1)
    # Step 2: Compute the (encoding) distance between the anchor and the negative, you will need to sum over axis=-1
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)),axis=-1)
    # Step 3: subtract the two previous distances and add alpha.
    basic_loss = tf.add(tf.subtract(pos_dist,neg_dist),alpha)
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))
    ### END CODE HERE ###
    
    return loss


In [3]:
### THIS USES PICKLE TO LOAD WEIGHTS FROM FACENET AND MODEL STORED IN FILE FRMODEL.P. COMPLETE FACENET MODEL WITH WEIGHTS USED
FRmodel = faceRecoModel(input_shape=(3, 96, 96))
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

pickle.dump(FRmodel,open("FRmodel.p","wb"))

  